<a href="https://colab.research.google.com/github/Shadman1077/Technonext/blob/main/TechnoNext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install related dependency

In [ ]:
!pip install -U accelerate
!pip install -U datasets

## import library

In [ ]:
import torch
from transformers import (
    BertTokenizer,
    BertTokenizerFast,
    BertForTokenClassification,
    AdamW,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification,
    AutoModelForTokenClassification,
)

In [ ]:
from torch.utils.data import DataLoader
from datasets import load_dataset
from tqdm import tqdm
import datasets
import numpy as np
from transformers import pipeline

## Load dataset

In [ ]:
dataset = load_dataset("tuetschek/atis")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

## Tokenize the dataframe

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
custom_tokens = ["o'clock", "st."]
tokenizer.add_tokens(custom_tokens)

2

In [ ]:
unique_labels = set()

In [ ]:
for example in dataset["train"]["slots"]:
    tokens = example.split(" ")
    for label in tokens:
        unique_labels.add(label)

for example in dataset["test"]["slots"]:
    tokens = example.split(" ")
    for label in tokens:
        unique_labels.add(label)

num_labels = len(unique_labels)
print(num_labels)

lookup_table = dict()
for i, x in enumerate(unique_labels):
    lookup_table[x] = i
label_list = {value: key for key, value in lookup_table.items()}

129


In [ ]:

def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["text"], truncation=True)
    labels = []

    for i, label in enumerate(examples["slots"]):
        label = label.split(" ")
        indexes = []
        temp_tokens = examples["text"][i].split(" ")
        for j in range(len(temp_tokens)):
            if temp_tokens[j][0] == "'":
                indexes.append(j)

        for k in range(len(indexes)):
            indexes[k] += k
            label.insert(indexes[k], "O")

        for j in range(len(label)):
            label[j] = lookup_table[label[j]]

        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)

            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_datasets = dataset["train"].map(tokenize_and_align_labels, batched=True)
tokenized_datasets_test = dataset["test"].map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/4978 [00:00<?, ? examples/s]

Map:   0%|          | 0/893 [00:00<?, ? examples/s]

In [ ]:
!pip install seqeval


In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)
metric = datasets.load_metric("seqeval")

<ipython-input-12-756e74dc04a5>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("seqeval")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:

def compute_metrics(eval_preds):
    pred_logits, labels = eval_preds

    pred_logits = np.argmax(pred_logits, axis=2)
    predictions = [
        [
            label_list[eval_preds]
            for (eval_preds, l) in zip(prediction, label)
            if l != -100
        ]
        for prediction, label in zip(pred_logits, labels)
    ]

    true_labels = [
        [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(pred_logits, labels)
    ]
    results = metric.compute(predictions=predictions, references=true_labels)

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Model building

In [ ]:
# eval_dataset=tokenized_datasets["validation"],
model = AutoModelForTokenClassification.from_pretrained(
    "bert-base-uncased", num_labels=num_labels
)
model.resize_token_embeddings(len(tokenizer))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(30524, 768)

In [ ]:
!pip install accelerate==0.20.1


In [ ]:
args = TrainingArguments(
    "test-ner",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=256,
    per_device_eval_batch_size=256,
    num_train_epochs=15,
    weight_decay=0.01,
)

# Training

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets_test,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.788605,0.391467,0.173466,0.240405,0.661294
2,No log,1.042359,0.538462,0.529353,0.533869,0.809968
3,No log,0.621152,0.781339,0.727695,0.753563,0.901909
4,No log,0.462150,0.789853,0.764179,0.776804,0.913043
5,No log,0.387290,0.804238,0.793035,0.798597,0.921951
6,No log,0.342067,0.822421,0.824876,0.823646,0.931495
7,No log,0.309811,0.838572,0.849420,0.843961,0.939979
8,No log,0.283719,0.845753,0.862023,0.853811,0.946341
9,No log,0.262828,0.859948,0.871642,0.865755,0.949099
10,No log,0.246341,0.875288,0.882255,0.878758,0.952492


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=300, training_loss=0.5244063313802083, metrics={'train_runtime': 432.4517, 'train_samples_per_second': 172.667, 'train_steps_per_second': 0.694, 'total_flos': 1363539481113204.0, 'train_loss': 0.5244063313802083, 'epoch': 15.0})

# Save the model

In [ ]:
model.save_pretrained("ner_model")
tokenizer.save_pretrained("tokenizer")
id2label = {str(i): label for i, label in enumerate(lookup_table)}
label2id = {label: str(i) for i, label in enumerate(lookup_table)}

In [ ]:
from transformers import pipeline
import json

In [ ]:
config = json.load(open("ner_model/config.json"))
config["id2label"] = id2label
config["label2id"] = label2id
json.dump(config, open("ner_model/config.json", "w"))

In [ ]:
model_fine_tuned = AutoModelForTokenClassification.from_pretrained("ner_model")
nlp = pipeline("ner", model=model_fine_tuned, tokenizer=tokenizer)

# Predict the output

In [ ]:
example = "I need to fly from Kansas City to Chicago leaving next Wednesday and returning the following day."

In [ ]:
ner_results = nlp(example)
for x in ner_results:
    print(x["entity"], x["word"])
    # print(type(x))

print("DONE")

B-fromloc.city_name kansas
I-fromloc.city_name city
B-toloc.city_name chicago
B-depart_date.date_relative next
B-depart_date.day_name wednesday
B-depart_date.date_relative following
DONE


In [ ]:
def extract_return_date_time(example):
    ner_results = nlp(example)
    return_entities = []
    for entity in ner_results:
        if entity["entity"] in ["B-return_date", "I-return_date", "B-return_time", "I-return_time"]:
            return_entities.append(entity)

    return return_entities


In [ ]:
# Function to find relevant return date and time information
def find_return_date_time(entities):
    for entity in entities:
        if "following day" in entity.lower() or "the following day" in entity.lower():
            return entity

In [ ]:
# Extract entities for return date and time
entities_1 = extract_return_date_time(example)

In [ ]:
return_info = find_return_date_time(entities_1)

In [ ]:
# Display the output
if return_info:
    print(return_info)
else:
  print("No return date/time detected.")

No return date/time detected.
